In [2]:
import cv2
import os
import pickle
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import SGDRegressor
from sklearn.metrics import r2_score, mean_squared_error

from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeCV
from sklearn.linear_model import ElasticNetCV
from sklearn.linear_model import LassoCV 

In [3]:
#Path to the raw images
path_raw = r"C:\Users\Sameed\Desktop\ML_Overall_Work\More_Improving_Age_Regression\Dataset\UTKFace (Reduced and sameed + arham)"

#Get the names of all the images
images = os.listdir(path_raw)

#Store the paths of all the cropped images we will save later
paths_cropped = []

img_dim = 32


In [4]:
unknown_ages = []
features = []

#Store all the paths for the unknown images
for unknowns in enumerate(images):
    
    age = ""
    for i in range(0,6):
        if(unknowns[1][i] == "_"):
            break
        else:
            age += unknowns[1][i]

    unknown_ages.append(int(age))

    path = path_raw + "\\" + unknowns[1]
    image = cv2.imread(path)

    img_resize = cv2.resize(image, (img_dim,img_dim))
    features.append(img_resize.flatten())


In [5]:
x = np.array(features)
y = np.array(unknown_ages)

x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.3)

In [13]:


param_grid = {
    "penalty" : ['l1', 'l2', 'elasticnet', None],
    "eta0" : [0.00001,0.0001,0.001,0.01,0.1],
    "loss" : ["squared_error","epsilon_insensitive","huber","squared_epsilon_insensitive"]
}

sgd_reg = SGDRegressor()
grid_search = GridSearchCV(sgd_reg, param_grid, cv=None, verbose=2)
grid_search.fit(x,y)

best_params = grid_search.best_params_
print(f"Best parameters: {best_params}")



Fitting 5 folds for each of 80 candidates, totalling 400 fits
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l1; total time=   8.6s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l1; total time=   8.2s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l1; total time=   7.2s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l1; total time=   5.0s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l1; total time=   4.3s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l2; total time=   1.5s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l2; total time=   1.5s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l2; total time=   1.6s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l2; total time=   2.0s
[CV] END .........eta0=1e-05, loss=squared_error, penalty=l2; total time=   1.2s
[CV] END .eta0=1e-05, loss=squared_error, penalty=elasticnet; total time=   5.2s
[CV] END .eta0=1e-05, loss=squared_error, penal

In [16]:
#Best parameters: {'eta0': 1e-05, 'loss': 'huber', 'penalty': 'elasticnet'}
print("Best parameters: {'eta0': 1e-05, 'loss': 'huber', 'penalty': 'elasticnet'}")


sgd_regressor_best = SGDRegressor(loss="huber",penalty="elasticnet", eta0=0.00001)
sgd_regressor_best.fit(x_train, y_train)


y_pred_sgd_test = sgd_regressor_best.predict(x_test)
y_pred_sgd_val = sgd_regressor_best.predict(x_train)

mse_test = mean_squared_error(y_test, y_pred_sgd_test)
r2_test = r2_score(y_test, y_pred_sgd_test)

mse_val = mean_squared_error(y_train, y_pred_sgd_val)
r2_val = r2_score(y_train, y_pred_sgd_val)

print("intercept=", sgd_regressor_best.intercept_)
print("coef=", sgd_regressor_best.coef_)

print(f"Testing: \n R2 = {r2_test} \n MSE = {mse_test} \n\n Validation: \n R2 = {r2_val} \n Mse = {mse_val}" )


Best parameters: {'eta0': 1e-05, 'loss': 'huber', 'penalty': 'elasticnet'}
intercept= [0.00017668]
coef= [ 1.86925527e-03  3.49269860e-03  2.52216876e-03 ... -1.56876892e-03
 -2.05656354e-05 -2.78842787e-03]
Testing: 
 R2 = 0.29457232759622687 
 MSE = 289.1994432175238 

 Validation: 
 R2 = 0.5292935074592306 
 Mse = 176.66010459543662


In [17]:

sgd_regressor = SGDRegressor()
sgd_regressor.fit(x_train, y_train)


y_pred_sgd_test = sgd_regressor.predict(x_test)
y_pred_sgd_val = sgd_regressor.predict(x_train)

mse_test = mean_squared_error(y_test, y_pred_sgd_test)
r2_test = r2_score(y_test, y_pred_sgd_test)

mse_val = mean_squared_error(y_train, y_pred_sgd_val)
r2_val = r2_score(y_train, y_pred_sgd_val)

print("intercept=", sgd_regressor.intercept_)
print("coef=", sgd_regressor.coef_)

print(f"Testing: \n R2 = {r2_test} \n MSE = {mse_test} \n\n Validation: \n R2 = {r2_val} \n Mse = {mse_val}" )

intercept= [2.02057265e+10]
coef= [-1.14815640e+12 -1.15861570e+12 -3.04693126e+12 ...  2.28172565e+12
  1.19453832e+12  3.58577793e+12]
Testing: 
 R2 = -1.035651136196906e+29 
 MSE = 4.24578937958515e+31 

 Validation: 
 R2 = -1.1636907481571606e+29 
 Mse = 4.367429227001371e+31


In [25]:

ridge = RidgeCV(alphas=[0.00001,0.0001,0.001,0.01,0.1])

ridge.fit(x, y)


y_pred_ridge_test = ridge.predict(x_test)
y_pred_ridge_val = ridge.predict(x_train)

mse_test = mean_squared_error(y_test, y_pred_ridge_test)
r2_test = r2_score(y_test, y_pred_ridge_test)

mse_val = mean_squared_error(y_train, y_pred_ridge_val)
r2_val = r2_score(y_train, y_pred_ridge_val)

print("Best alpha:", ridge.alpha_)
print("intercept=", ridge.intercept_)
print("coef=", ridge.coef_)

print(f"Testing: \n R2 = {r2_test} \n MSE = {mse_test} \n\n Validation: \n R2 = {r2_val} \n Mse = {mse_val}" )



Best alpha: 0.1
intercept= 36.26796266134528
coef= [ 0.16289873 -0.1838536   0.0060522  ... -0.07434907  0.15201617
 -0.02620253]
Testing: 
 R2 = 0.9999999995863119 
 MSE = 1.6959690617641762e-07 

 Validation: 
 R2 = 0.9997056573759304 
 Mse = 0.1104692617991223


In [26]:

lasso = LassoCV(alphas=[0.00001,0.0001,0.001,0.01,0.1])

lasso.fit(x, y)

y_pred_lasso_test = lasso.predict(x_test)
y_pred_lasso_val = lasso.predict(x_train)

mse_test = mean_squared_error(y_test, y_pred_lasso_test)
r2_test = r2_score(y_test, y_pred_lasso_test)

mse_val = mean_squared_error(y_train, y_pred_lasso_val)
r2_val = r2_score(y_train, y_pred_lasso_val)

print("Best alpha:", lasso.alpha_)
print("intercept=", lasso.intercept_)
print("coef=", lasso.coef_)

print(f"Testing: \n R2 = {r2_test} \n MSE = {mse_test} \n\n Validation: \n R2 = {r2_val} \n Mse = {mse_val}" )



c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.507e+04, tolerance: 6.052e+01
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 3.306e+03, tolerance: 6.052e+01
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 7.263e+02, tolerance: 6.052e+01
  model = cd_fast.enet_

Best alpha: 0.1
intercept= 36.90205533030036
coef= [ 0.04658213 -0.07383511  0.03491625 ... -0.03334419  0.08342905
 -0.02869899]
Testing: 
 R2 = 0.9595304655752072 
 MSE = 16.591023120827465 

 Validation: 
 R2 = 0.9558670546849198 
 Mse = 16.5634654694963


c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.896e+04, tolerance: 8.125e+01
  model = cd_fast.enet_coordinate_descent(


In [27]:

elastic = ElasticNetCV(alphas=[0.00001,0.0001,0.001,0.01,0.1])

elastic.fit(x, y)

y_pred_elastic_test = elastic.predict(x_test)
y_pred_elastic_val = elastic.predict(x_train)

mse_test = mean_squared_error(y_test, y_pred_elastic_test)
r2_test = r2_score(y_test, y_pred_elastic_test)

mse_val = mean_squared_error(y_train, y_pred_elastic_val)
r2_val = r2_score(y_train, y_pred_elastic_val)

print("Best alpha:", elastic.alpha_)
print("intercept=", elastic.intercept_)
print("coef=", elastic.coef_)

print(f"Testing: \n R2 = {r2_test} \n MSE = {mse_test} \n\n Validation: \n R2 = {r2_val} \n Mse = {mse_val}" )




c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 1.167e+04, tolerance: 6.052e+01
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.240e+03, tolerance: 6.052e+01
  model = cd_fast.enet_coordinate_descent(
c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 5.410e+02, tolerance: 6.052e+01
  model = cd_fast.enet_

Best alpha: 0.1
intercept= 37.283125135252135
coef= [ 0.04234396 -0.07627742  0.04282558 ... -0.03048236  0.08887105
 -0.03369789]
Testing: 
 R2 = 0.9675068316575418 
 MSE = 13.32100590977883 

 Validation: 
 R2 = 0.9632593880518443 
 Mse = 13.789060598307701


c:\Python311\Lib\site-packages\sklearn\linear_model\_coordinate_descent.py:631: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations, check the scale of the features or consider increasing regularisation. Duality gap: 2.330e+04, tolerance: 8.125e+01
  model = cd_fast.enet_coordinate_descent(


In [29]:
ols = LinearRegression()
ols.fit(x_train, y_train)


y_pred_ols_test = ols.predict(x_test)
y_pred_ols_val = ols.predict(x_train)

mse_test = mean_squared_error(y_test, y_pred_ols_test)
r2_test = r2_score(y_test, y_pred_ols_test)

mse_val = mean_squared_error(y_train, y_pred_ols_val)
r2_val = r2_score(y_train, y_pred_ols_val)

print("Intercept =", ols.intercept_)
print("Coef =", ols.coef_)

print(f"Testing: \n R2 = {r2_test} \n MSE = {mse_test} \n\n Validation: \n R2 = {r2_val} \n Mse = {mse_val}" )


Intercept = -1491147386.0275977
Coef = [ -5577736.17283156 -25315870.41436087  51127113.75116897 ...
  -9119928.6615228   15413030.38322135  17892538.55384046]
Testing: 
 R2 = -5.184394522872705e+17 
 MSE = 2.1254113895557284e+20 

 Validation: 
 R2 = 0.9997056575655837 
 Mse = 0.11046919062063132


MODEL SCORES 
------------

SGD REGRESSOR:

Vanilla: 

intercept= [2.02057265e+10]

coef= [-1.14815640e+12 -1.15861570e+12 -3.04693126e+12 ...  2.28172565e+12
  1.19453832e+12  3.58577793e+12]

Testing: 

 R2 = -1.035651136196906e+29 

 MSE = 4.24578937958515e+31 


 Validation: 

 R2 = -1.1636907481571606e+29 

 Mse = 4.367429227001371e+31

 -----------------------------------
 

Best parameters: {'eta0': 1e-05, 'loss': 'huber', 'penalty': 'elasticnet'}

intercept= [0.00017668]

coef= [ 1.86925527e-03  3.49269860e-03  2.52216876e-03 ... -1.56876892e-03
 -2.05656354e-05 -2.78842787e-03]

Testing: 

 R2 = 0.29457232759622687 

 MSE = 289.1994432175238 

 Validation: 

 R2 = 0.5292935074592306 

 Mse = 176.66010459543662


--------------------------------------------------------------------------
RidgeCV

Best alpha: 0.1

intercept= 36.26796266134528

coef= [ 0.16289873 -0.1838536   0.0060522  ... -0.07434907  0.15201617
 -0.02620253]

Testing: 

 R2 = 0.9999999995863119 

 MSE = 1.6959690617641762e-07 


 Validation: 

 R2 = 0.9997056573759304 

 Mse = 0.1104692617991223

----------------------------------
LassoCV

Best alpha: 0.1

intercept= 36.90205533030036

coef= [ 0.04658213 -0.07383511  0.03491625 ... -0.03334419  0.08342905
 -0.02869899]

Testing: 

 R2 = 0.9595304655752072 

 MSE = 16.591023120827465 


 Validation: 

 R2 = 0.9558670546849198 

 Mse = 16.5634654694963

------------------------------
ElasticNetCV

Best alpha: 0.1

intercept= 37.283125135252135

coef= [ 0.04234396 -0.07627742  0.04282558 ... -0.03048236  0.08887105
 -0.03369789]

Testing: 

 R2 = 0.9675068316575418 

 MSE = 13.32100590977883 


 Validation: 

 R2 = 0.9632593880518443 

 Mse = 13.789060598307701

-----------------------
OLS

Intercept = -1491147386.0275977

Coef = [ -5577736.17283156 -25315870.41436087  51127113.75116897 ...
  -9119928.6615228   15413030.38322135  17892538.55384046]

Testing: 

 R2 = -5.184394522872705e+17 

 MSE = 2.1254113895557284e+20 

 Validation: 

 R2 = 0.9997056575655837 

 Mse = 0.11046919062063132

In [30]:
#Choosing the best model

#RidgeCV
pickle.dump(ridge, open('ridge.pkl', 'wb'))
